# This script compares the shape of a group of samples
## Input a dictionary with the location of the samples and their name.
## Also, add the name of the histogram and the binning.

In [75]:
from histogramHelpers import normalization
import ROOT as r
import numpy as np

rootDirectory = "/Users/diegomac/Documents/HEP/VBF-Analysis/Zll/Z-peak/SR/"
## QCDjj shapes ###
if (True):
    channel = "Zll_"
    samples = {"MadGraph@NLO":rootDirectory+channel+"MGNLORW.root",
               "MadGraph@LO": rootDirectory+channel+"MGRW.root",
               "MadGraph@LO-FLAT": rootDirectory+channel+"MGRWFlat.root",
               "Sherpa2.2.11": rootDirectory+channel+"SherpaNLORW.root",
               "Sherpa2.2.1": rootDirectory+channel+"SherpaRW.root",
              "Sherpa2.2.1-FLAT": rootDirectory+channel+"SherpaRWFlat.root"}

    colors = {"MadGraph@NLO":r.kBlue,"MadGraph@LO": r.kGreen,"MadGraph@LO-FLAT": r.kViolet,
              "Sherpa2.2.11": r.kRed,"Sherpa2.2.1": r.kBlack,"Sherpa2.2.1-FLAT": r.kYellow}

    styles = {"MadGraph@NLO":1,"MadGraph@LO": 1,"MadGraph@LO-FLAT": 1,
              "Sherpa2.2.11": 1,"Sherpa2.2.1":1,"Sherpa2.2.1-FLAT": 1}
    
## EWjj shapes ###
if (False):

    channel = "Signal_"
    samples = {"PowHeg+Pythia8":rootDirectory+channel+"PoPy.root",
               "Sherpa2.2.11": rootDirectory+channel+"Sherpa.root",}

    colors = {"PowHeg+Pythia8":r.kBlue,
               "Sherpa2.2.11": r.kBlack}

    styles = {"PowHeg+Pythia8":1,
           "Sherpa2.2.11": 1}
    
    

histogramName = "mass_jj"
bining = np.array([   0.,  250.,  500.,  750., 1000., 1250., 1500., 2000., 2500., 3000., 4000., 5000.])
normalisationConstant = 250.0

defaultSample = "Sherpa2.2.1"

In [94]:
r.TH1.AddDirectory(False)
r.gStyle.SetOptStat(0)

canvas = r.TCanvas("canvas")
canvas.cd()

pad1 = r . TPad (" pad1 "," pad1 " ,0 ,0.45 ,1 ,1)
pad1.SetTopMargin(0.03)
pad1.SetRightMargin(0.03)
pad1.SetLeftMargin(0.08)
pad1.SetBottomMargin(0.0)
pad1.Draw ()
pad1.cd ()
pad1.SetLogy()
pad1.cd()



for sampleName,sampleFile in samples.items():

    file = r.TFile.Open(sampleFile,"READ")
    histogram = file.Get(histogramName)
    histogram.SetDirectory(0)
    histogram = histogram.Rebin(len(bining)-1,sampleName,bining)
    
    integral = histogram.Integral(1,-1)
    histogram.Scale(1.0/integral)
    
    histogram.SetLineColor(colors[sampleName])
    histogram.SetLineStyle(styles[sampleName])
    histogram.SetTitle(sampleName)
    histogram.GetYaxis().SetRangeUser(0.00005,0.6)
    
    normalization([histogram],normalisationConstant)
    histogram.Draw("HIST same")
    
pad1.BuildLegend()

pad1.FindObject("MadGraph@NLO").SetTitle("QCDjj Shapes")
pad1.FindObject("MadGraph@NLO").SetTitleOffset(0.2)
pad1.Update()

canvas.cd()
pad2 = r . TPad (" pad2 "," pad2 " ,0 ,0.10 ,1 ,0.45)
pad2.SetRightMargin(0.03)
pad2.SetLeftMargin(0.08)
pad2.SetTopMargin(0)
pad2.SetBottomMargin(0.18)
pad2.Draw ()
pad2.cd()

#Construct default sample
file = r.TFile.Open(samples[defaultSample],"READ")
defaultHistogram = file.Get(histogramName)
defaultHistogram.SetDirectory(0)
defaultHistogram = defaultHistogram.Rebin(len(bining)-1,samples[defaultSample],bining)
integral = defaultHistogram.Integral(1,-1)
defaultHistogram.Scale(1.0/integral)
normalization([defaultHistogram],normalisationConstant)
file.Close()

dummy = defaultHistogram.Clone()
dummy.Divide(dummy)
dummy.SetTitle("")
dummy.Draw("LF2 same")

dummy.GetYaxis().SetRangeUser(0.1,1.5)


for sampleName,sampleFile in samples.items():
    
    if sampleName!=defaultSample:

        file = r.TFile.Open(sampleFile,"READ")
        histogram = file.Get(histogramName)
        histogram.SetDirectory(0)
        histogram = histogram.Rebin(len(bining)-1,sampleName,bining)

        integral = histogram.Integral(1,-1)
        print(integral)
        histogram.Scale(1.0/integral)

        histogram.SetLineColor(colors[sampleName])
        histogram.SetFillColorAlpha(colors[sampleName],0.0)
        histogram.SetLineStyle(styles[sampleName])
        histogram.SetTitle(sampleName)

        normalization([histogram],normalisationConstant)
        
        histogram.Divide(defaultHistogram)
        histogram.Draw("e1 X0 HIST L P same")
        histogram.Draw("HIST L P same")

        
dummy.GetYaxis().SetTitle("PowHeg+Pythia8/"+defaultSample)
dummy. GetYaxis (). SetTitleSize (0.07)
dummy. GetYaxis (). SetTitleOffset (.25)

dummy.GetXaxis().SetTitle("Mjj(GeV)")
dummy. GetXaxis (). SetTitleSize (0.10)
dummy. GetXaxis (). SetTitleOffset (0.85)
dummy. GetXaxis (). SetLabelSize(0.07)


canvas.Print("QCDjj.pdf")
    

22608.9996231501
23817.70100569725
23792.078945457935
22613.50568008423
23569.6916577816


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file QCDjj.pdf has been created


# This script takes a TTree and creates a histogram with one of the leafs.

In [76]:
import ROOT as r
import numpy as np

rootDirectory = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/High-Mass/MVATrainingData/"

samplesBG = {#"Higgs-NOTVBF":rootDirectory+"Higgs.root",
           #"ttbar": rootDirectory+"ttbar.root",
           #"Z+jets": rootDirectory+"Zjets.root",
           #"Single-top": rootDirectory+"singletop.root",
           #"Sherpa2.2.1": rootDirectory+"Ztautau_SherpaRW.root",
          #"MG": rootDirectory+"Ztautau_MGRW.root",
             #"VV": rootDirectory+"VV.root",
             #"VV_EWK": rootDirectory+"VV_EWK.root",
             #"W_EWK-Sherpa": rootDirectory+"W_EWK_Sherpa.root",
             #"W_EWK-PoPy": rootDirectory+"W_EWK_PoPy.root"
            "Higgs-VBF":rootDirectory+"Higgs.root",
            "Z-VBF-Sherpa":rootDirectory+"Signal_Sherpa.root",
            "Z-VBF-PoPy":rootDirectory+"Signal_PoPy.root"
            }

colorsBG = {"Higgs-NOTVBF":r.kBlack,
           "ttbar": r.kRed,
           "Z+jets": r.kGreen,
           "Single-top": r.kViolet,
           "Sherpa2.2.1": r.kYellow+3,
          "MG": r.kCyan,
             "VV": r.kBlue,
             "VV_EWK": r.kRed,
             "W_EWK-Sherpa": r.kGreen,
             "W_EWK-PoPy": r.kYellow+6,
            "Higgs-VBF": r.kRed,
            "Z-VBF-Sherpa":r.kGreen,
            "Z-VBF-PoPy": r.kBlue,
            }

histogramName = "mcWeight"

In [79]:
r.TH1.AddDirectory(False)

canvas = r.TCanvas("canvas")
canvas.cd()
canvas.SetLogy()

directory = r.TDirectory("default","default")
histos = []*len(samplesBG) # Trick to save each histogram in different part of memory

legend = r . TLegend (0.45 ,0.55 ,0.85 ,0.75)

for sample in samplesBG:
    file = r.TFile.Open(samplesBG[sample],"READ")
    print(2,r.gDirectory)
    tree = file.Get("SIGNAL")
    print(3,r.gDirectory)
    tree.Draw(histogramName+">>"+sample+"(80,-0.5,1.5)")
    
    histo = r.gPad.GetPrimitive(sample)
    histos.append(histo)
    histo.SetDirectory(0)
    print(4,r.gDirectory)
    file.Close()
    print(5,r.gDirectory)
    legend.AddEntry(sample,sample,"f")
    histo.SetLineColor(colorsBG[sample])
        
for i in histos:
    i.Draw("sames")
    i.GetYaxis().SetRangeUser(0.1,100000)
    
legend.SetNColumns(3)
r.gStyle.SetLegendBorderSize(0)
legend . SetLineWidth (0)
legend . Draw ()

canvas.Print("TEST.pdf")
print(8,r.gDirectory)
canvas.Update()

temp = directory.GetListOfKeys()
print(temp)
for i in temp:
    print(i)

2 <cppyy.gbl.TDirectory* object at 0x107028b40>
3 <cppyy.gbl.TDirectory* object at 0x107028b40>
4 <cppyy.gbl.TDirectory* object at 0x107028b40>
5 <cppyy.gbl.TDirectory* object at 0x107918ee0>
2 <cppyy.gbl.TDirectory* object at 0x125efee80>
3 <cppyy.gbl.TDirectory* object at 0x125efee80>
4 <cppyy.gbl.TDirectory* object at 0x125efee80>
5 <cppyy.gbl.TDirectory* object at 0x107918ee0>
2 <cppyy.gbl.TDirectory* object at 0x1448e7cd0>
3 <cppyy.gbl.TDirectory* object at 0x1448e7cd0>
4 <cppyy.gbl.TDirectory* object at 0x1448e7cd0>
5 <cppyy.gbl.TDirectory* object at 0x107918ee0>
8 <cppyy.gbl.TDirectory* object at 0x107918ee0>


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file TEST.pdf has been created
